In [47]:
import pandas as pd
import scipy.stats as ss

import plotly.express as px

Начнём с импорта и предварительного анализа данных

In [48]:
dating = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-sedelkin/analytics_practice/statistics/data/dating_data.csv')

In [59]:
dating

,user_id_1,user_id_2,group,is_match
0,79,91,new,match
1,716,353,new,match
2,423,677,old,no_match
3,658,165,new,match
4,969,155,old,match
...,...,...,...,...
14509,476,631,old,no_match
14510,455,770,new,no_match
14511,260,6,new,match
14512,885,812,new,match


In [50]:
dating.isna().sum()

user_id_1    0
user_id_2    0
group        0
is_match     0
dtype: int64

In [51]:
dating.nunique()

user_id_1    1000
user_id_2    1000
group           2
is_match        2
dtype: int64

Далее, для удобства работы с переменными, которые по сути своей являются категориальными, заменим 0 и 1 в соответствующих столбцах на название групп тестирования и результат взаимодействия пользователей

In [53]:
dating.group = dating.group.apply(lambda x: 'old' if x == 0 else 'new')
dating.is_match = dating.is_match.apply(lambda x: 'no_match' if x == 0 else 'match')

In [54]:
dating.head()

,user_id_1,user_id_2,group,is_match
0,79,91,new,match
1,716,353,new,match
2,423,677,old,no_match
3,658,165,new,match
4,969,155,old,match


Визуализируем различия между двумя группами

In [55]:
px.histogram(dating[['group', 'is_match']], x='group', color='is_match')

In [56]:
pd.crosstab(dating.group, dating.is_match, normalize=True)

is_match,match,no_match
group,,
new,0.269326,0.400510
old,0.064352,0.265812


По графику можно заключить, что доля успешных взаимодействий (match) в группе пользователей с новым алгоритмом выше, однако необходимо проверить, что данное отличие статистически значимо. 

Так как сравниваются две категориальных переменных, используем тест "хи-квадрат".

H0 – взаимосвязь между количеством match и тем, какой алгоритм применялся у пользователя (старый или новый) отсутствует
H1 - данная взаимосвязь есть

In [57]:
from scipy.stats import chi2_contingency, chi2
stat, p, dof, expected = chi2_contingency(pd.crosstab(dating.group, dating.is_match))

In [58]:
stat, p

(618.6889141576198, 1.4418299163662586e-136)

По результатам теста видим, что обнаружена значимая взаимосвязь, а p-value существенно ниже 0,05, следовательно данные различия статистически-значимы.

**Вывод: по итогам теста рекомендуется вводить новый алгоритм.**